<a href="https://colab.research.google.com/github/RamadeviKN/machine-learning-zoomcamp/blob/main/homework6_decisiontrees_ensemblelearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Ensemble Learning

# Students Performance in 2024 JAMB

# Importing the libraries

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Data cleaning and preparation

1.   Downloading the dataset
2.   Re-encoding the categorical variables
3.  Doing the train/validation/test split



In [26]:
data = 'https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv'
!wget $data

--2024-11-04 20:18:59--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-11-04 20:18:59--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: ‘jamb_exam_results.csv.1’

jamb_exam_results.c 100%[===================>] 382.33K  --.-KB/s    in 0.06s   

2024-11-04 20:18:59 (6.76 MB/s) - ‘jamb_exam_resu

In [27]:
!head jamb_exam_results.csv

JAMB_Score,Study_Hours_Per_Week,Attendance_Rate,Teacher_Quality,Distance_To_School,School_Type,School_Location,Extra_Tutorials,Access_To_Learning_Materials,Parent_Involvement,IT_Knowledge,Student_ID,Age,Gender,Socioeconomic_Status,Parent_Education_Level,Assignments_Completed
192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,None,1
182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1
202,25,85,2,13.6,Public,Urban,Yes,No,Medium,Low,6,15,Male,Low,Tertiary,1
251,35,85,4,2.6,Public,Urban,No,Yes,Low,Medium,7,16,Female,Medium,Primary,4
129,27,75,3,9.4,Public,Urban,No,Yes,Low,Medium,8,19,Female,Low,Tertiary,3
220,23,85,3,4.6,Public,Rural,No,No,Low,Medium,9,19,Female,Medium,Tertiary,1


In [28]:
df = pd.read_csv(data)

In [29]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [30]:
df = df.drop('student_id', axis=1)

In [56]:
print(df.columns)

Index(['jamb_score', 'study_hours_per_week', 'attendance_rate',
       'teacher_quality', 'distance_to_school', 'school_type',
       'school_location', 'extra_tutorials', 'access_to_learning_materials',
       'parent_involvement', 'it_knowledge', 'age', 'gender',
       'socioeconomic_status', 'parent_education_level',
       'assignments_completed'],
      dtype='object')


In [31]:
df.describe().round()

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,age,assignments_completed
count,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0
mean,174.0,20.0,84.0,3.0,10.0,18.0,2.0
std,48.0,10.0,9.0,1.0,5.0,2.0,1.0
min,100.0,0.0,50.0,1.0,0.0,15.0,1.0
25%,135.0,13.0,78.0,2.0,7.0,16.0,1.0
50%,170.0,19.0,84.0,2.0,10.0,18.0,1.0
75%,209.0,26.0,91.0,3.0,13.0,20.0,2.0
max,367.0,40.0,100.0,5.0,20.0,22.0,5.0


Explore the missing column in dataframe

In [32]:
missing_values = df.isnull().any()
print(missing_values)


jamb_score                      False
study_hours_per_week            False
attendance_rate                 False
teacher_quality                 False
distance_to_school              False
school_type                     False
school_location                 False
extra_tutorials                 False
access_to_learning_materials    False
parent_involvement              False
it_knowledge                    False
age                             False
gender                          False
socioeconomic_status            False
parent_education_level           True
assignments_completed           False
dtype: bool


In [33]:
missing_columns = missing_values[missing_values == True].index
print("Columns with missing values:", list(missing_columns))

Columns with missing values: ['parent_education_level']


In [34]:
#Filling missing column values with 0
df['parent_education_level'] = df['parent_education_level'].fillna(0)

Do train/validation/test split with 60%/20%/20% distribution.

In [35]:
from sklearn.model_selection import train_test_split

# Step 1: Split 20% of the data as test set
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

# Step 2: Split the remaining 80% (df_full_train) into train (60%) and validation (20%) sets
# Since df_full_train is 80% of the original data, setting test_size=0.25 here gives us 20% of the original data for validation
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)


In [36]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [37]:
y_train = df_train['jamb_score'].values
y_val = df_val['jamb_score'].values
y_test = df_test['jamb_score'].values

In [38]:
del df_train['jamb_score']
del df_val['jamb_score']
del df_test['jamb_score']

Use DictVectorizer(sparse=True) to turn the dataframes into matrices.

In [39]:
from sklearn.feature_extraction import DictVectorizer
train_dicts = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)

In [52]:
from sklearn.tree import DecisionTreeClassifier
dtModel = DecisionTreeClassifier()
dtModel.fit(X_train, y_train)

DecisionTreeClassifier()

In [53]:
print("X_train :")
print(X_train)
print("y_train :")
print(y_train)

X_train :
  (0, 1)	1.0
  (0, 2)	21.0
  (0, 3)	3.0
  (0, 4)	72.0
  (0, 5)	4.4
  (0, 6)	1.0
  (0, 8)	1.0
  (0, 11)	1.0
  (0, 13)	0.0
  (0, 19)	1.0
  (0, 21)	1.0
  (0, 23)	1.0
  (0, 25)	1.0
  (0, 27)	20.0
  (0, 28)	3.0
  (1, 1)	1.0
  (1, 2)	22.0
  (1, 3)	1.0
  (1, 4)	80.0
  (1, 5)	3.3
  (1, 7)	1.0
  (1, 8)	1.0
  (1, 10)	1.0
  (1, 15)	1.0
  (1, 19)	1.0
  :	:
  (2998, 6)	1.0
  (2998, 9)	1.0
  (2998, 10)	1.0
  (2998, 15)	1.0
  (2998, 17)	1.0
  (2998, 20)	1.0
  (2998, 23)	1.0
  (2998, 25)	1.0
  (2998, 27)	25.0
  (2998, 28)	2.0
  (2999, 1)	1.0
  (2999, 2)	17.0
  (2999, 3)	1.0
  (2999, 4)	67.0
  (2999, 5)	10.5
  (2999, 7)	1.0
  (2999, 8)	1.0
  (2999, 12)	1.0
  (2999, 15)	1.0
  (2999, 18)	1.0
  (2999, 21)	1.0
  (2999, 23)	1.0
  (2999, 24)	1.0
  (2999, 27)	18.0
  (2999, 28)	2.0
y_train :
[145 231 161 ... 216 234 103]


In [48]:
print(df_train.columns)

Index(['study_hours_per_week', 'attendance_rate', 'teacher_quality',
       'distance_to_school', 'school_type', 'school_location',
       'extra_tutorials', 'access_to_learning_materials', 'parent_involvement',
       'it_knowledge', 'age', 'gender', 'socioeconomic_status',
       'parent_education_level', 'assignments_completed'],
      dtype='object')


In [59]:
# Get the feature importances and the feature names
feature_importances = dtModel.tree_.feature
feature_names = dv.get_feature_names_out()

# Identify the feature used for the first split
print(feature_importances)
print(feature_names)
first_split_feature_index = feature_importances[0]

if first_split_feature_index != -2:  # -2 means no split
    first_split_feature_name = feature_names[first_split_feature_index]
    print("Feature used for splitting the data:", first_split_feature_name)
else:
    print("No feature was used for splitting.")

[27  4  0 ... -2 -2 -2]
['access_to_learning_materials=No' 'access_to_learning_materials=Yes'
 'age' 'assignments_completed' 'attendance_rate' 'distance_to_school'
 'extra_tutorials=No' 'extra_tutorials=Yes' 'gender=Female' 'gender=Male'
 'it_knowledge=High' 'it_knowledge=Low' 'it_knowledge=Medium'
 'parent_education_level' 'parent_education_level=Primary'
 'parent_education_level=Secondary' 'parent_education_level=Tertiary'
 'parent_involvement=High' 'parent_involvement=Low'
 'parent_involvement=Medium' 'school_location=Rural'
 'school_location=Urban' 'school_type=Private' 'school_type=Public'
 'socioeconomic_status=High' 'socioeconomic_status=Low'
 'socioeconomic_status=Medium' 'study_hours_per_week' 'teacher_quality']
Feature used for splitting the data: study_hours_per_week


Question 1: Which feature is used for splitting the data?

1. study_hours_per_week
2. attendance_rate
3. teacher_quality
4. distance_to_school

Answer : 1. study_hours_per_week

Train the Random Forest Regressor

In [60]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [63]:
# Fit the validation data
X_val = dv.transform(df_val.to_dict(orient='records'))

# Train a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Make predictions on the validation data
y_val_pred = rf_model.predict(X_val)

# Calculate RMSE
rmse = mean_squared_error(y_val, y_val_pred, squared=False)
print("RMSE on validation data:", rmse)

RMSE on validation data: 42.13724207871227


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Question 2: What's the RMSE of this model on the validation data?

1. 22.13
2. 42.13
3. 62.13
4. 82.12

Answer : 2. 42.13

In [64]:
# Step 1: Experiment with n_estimators
rmse_results = []

for n_estimators in range(10, 201, 10):
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=1, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    y_val_pred = rf_model.predict(X_val)
    rmse = mean_squared_error(y_val, y_val_pred, squared=False)
    rmse_results.append((n_estimators, rmse))
    print(f'n_estimators: {n_estimators}, RMSE: {rmse:.3f}')

# Step 2: Analyze RMSE results
# To find the point where RMSE stops improving significantly
for i in range(1, len(rmse_results)):
    if abs(rmse_results[i][1] - rmse_results[i-1][1]) < 0.001:  # Set a threshold for improvement
        print(f'RMSE stopped improving after n_estimators = {rmse_results[i-1][0]}')
        break

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


n_estimators: 10, RMSE: 42.137


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


n_estimators: 20, RMSE: 41.461


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


n_estimators: 180, RMSE: 40.641


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


n_estimators: 190, RMSE: 40.631
n_estimators: 200, RMSE: 40.601


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Question 3: After which value of n_estimators does RMSE stop improving? Consider 3 decimal places for calculating the answer.

1. 10
2. 25
3. 80
4. 200

Answer : 3. 80

In [65]:
# Step 1: Try different values for max_depth and n_estimators
max_depth_values = [10, 15, 20, 25]
n_estimators_range = range(10, 201, 10)
mean_rmse_results = {}

for max_depth in max_depth_values:
    rmse_list = []

    for n_estimators in n_estimators_range:
        rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=1, n_jobs=-1)
        rf_model.fit(X_train, y_train)
        y_val_pred = rf_model.predict(X_val)
        rmse = mean_squared_error(y_val, y_val_pred, squared=False)
        rmse_list.append(rmse)

    mean_rmse = sum(rmse_list) / len(rmse_list)
    mean_rmse_results[max_depth] = mean_rmse
    print(f'max_depth: {max_depth}, Mean RMSE: {mean_rmse:.3f}')

# Step 2: Determine the best max_depth based on mean RMSE
best_max_depth = min(mean_rmse_results, key=mean_rmse_results.get)
print(f'Best max_depth: {best_max_depth}, Mean RMSE: {mean_rmse_results[best_max_depth]:.3f}')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

max_depth: 10, Mean RMSE: 40.392


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

max_depth: 15, Mean RMSE: 40.735


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

max_depth: 20, Mean RMSE: 40.740


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

max_depth: 25, Mean RMSE: 40.788
Best max_depth: 10, Mean RMSE: 40.392


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Question 4: What's the best max_depth, using the mean RMSE?

1. 10
2. 15
3. 20
4. 25

Answer : 1. 10

Most important feature using the Random Forest model

In [68]:
# Train the Random Forest model with specified parameters
rf_model = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Get feature importances
feature_importances = rf_model.feature_importances_
feature_names = dv.get_feature_names_out()

# Create a DataFrame to display feature importances
importances_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
importances_df = importances_df.sort_values(by='importance', ascending=False)

# Display the most important features
print(importances_df.head(1))

                 feature  importance
27  study_hours_per_week    0.248354


Question 5: What's the most important feature (among these 4)?

1. study_hours_per_week
2. attendance_rate
3. distance_to_school
4. teacher_quality

Answer : 1. study_hours_per_week

In [69]:
!pip install xgboost

In [70]:
import xgboost as xgb

In [71]:
# Convert the data into DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [75]:
# Define XGBoost parameters and train model with eta=0.3
xgb_params_03 = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

# Train the model for 100 rounds with eta=0.3
model_xgbtest03 = xgb.train(xgb_params_03, dtrain, num_boost_round=100, evals=watchlist, early_stopping_rounds=10, verbose_eval=False)

In [76]:
# Calculate RMSE for eta=0.3
y_val_pred_03 = model_xgbtest03.predict(dval)
rmse_03 = mean_squared_error(y_val, y_val_pred_03, squared=False)
print(f'RMSE with eta=0.3: {rmse_03:.3f}')

RMSE with eta=0.3: 41.228


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [77]:
# Define XGBoost parameters and train model with eta=0.1
xgb_params_01 = xgb_params_03.copy()
xgb_params_01['eta'] = 0.1

# Train the model for 100 rounds with eta=0.1
model_xgbtest01 = xgb.train(xgb_params_01, dtrain, num_boost_round=100, evals=watchlist, early_stopping_rounds=10, verbose_eval=False)

In [78]:
# Calculate RMSE for eta=0.1
y_val_pred_01 = model_xgbtest01.predict(dval)
rmse_01 = mean_squared_error(y_val, y_val_pred_01, squared=False)
print(f'RMSE with eta=0.1: {rmse_01:.3f}')

RMSE with eta=0.1: 40.200


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Question 6: Which eta leads to the best RMSE score on the validation dataset?

1. 0.3
2. 0.1
3. Both give equal value

Answer :  2. 0.1 (As RMSE of eta=0.1 gives lesser RMSE value)